In [ ]:
from __future__ import print_function
import random
import matplotlib.pyplot as plt
import scipy.ndimage
import numpy as np, h5py
import os, time, sys
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import BatchNormalization, Convolution2D, Input , SeparableConv2D, Conv2D, Conv3D, Reshape
from keras.layers.core import Activation, Layer
from keras.callbacks import History, EarlyStopping, ModelCheckpoint, CSVLogger
history = History()
from itertools import cycle
from sklearn import metrics
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.backend.tensorflow_backend import set_session

In [ ]:
# Load data
f_data = '.../UNMIX-ME_CNN/trainD' # Training data directory
stacks = os.listdir(f_data)
numS = int(len(stacks))

# Pre-allocate for TPSF spectral data (time, x, y, channels)
tp = 256
x = 16
y = 16
ch = 16
tpsfD = np.ndarray(
        (numS, int(256), int(x), int(y), int(ch)), dtype=np.float32
        )
# Pre-allocate for abundance coefficients (time, x, y, channels)
nC = 4 # Number of coefficients
c = np.ndarray(
        (numS, int(nC), int(x), int(y)), dtype=np.float32
        )

i = 0;
for d in stacks:
    # Save values to respective mapping
    f = h5py.File(os.path.join(f_data,d),'r') 
    tpsfD[i,:,:,:,:] = f.get('tpsfs')
    f = h5py.File(os.path.join(f_data,d),'r') 
    c[i,:,:,:] = f.get('c')
    i = i + 1

# Move data to correct dimensionality/profile
tpsfD =  np.moveaxis(tpsfD, 1, -1)
tpsfD = np.moveaxis(tpsfD,1,-1)
c = np.moveaxis(c, 1, -1)

In [ ]:
print(tpsfD.shape)
print(c.shape)

In [ ]:
# resBlock with batch-normalization
def resblock_2D_BN(num_filters, size_filter, x):
    Fx = Conv2D(num_filters, size_filter, padding='same', activation=None)(x)
    Fx = BatchNormalization()(Fx)
    Fx = Activation('relu')(Fx)
    Fx = Conv2D(num_filters, size_filter, padding='same', activation=None)(Fx)
    Fx = BatchNormalization()(Fx)
    output = add([Fx, x])
    output = Activation('relu')(output)
    return output

# xCeptionBlock with batch-normalization
def xCeptionblock_2D_BN(num_filters, size_filter, x):
    Fx = SeparableConv2D(num_filters, size_filter, padding='same', activation=None)(x)
    Fx = BatchNormalization()(Fx)
    Fx = Activation('relu')(Fx)
    Fx = SeparableConv2D(num_filters, size_filter, padding='same', activation=None)(Fx)
    Fx = BatchNormalization()(Fx)
    output = add([Fx, x])
    output = Activation('relu')(output)
    return output

In [ ]:
modelD = None
xX = 16;
yY = 16;

t_data = Input(shape=(xX, yY, 256, 16))
tpsf = t_data

# Shared branch (4D input)
tpsf = Conv3D(128,kernel_size=(1,1,16),strides=(1,1,8), padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)
tpsf = Conv3D(64,kernel_size=(1,1,8),strides=(1,1,4), padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)
# Shared branch (4D ---> 3D) ** EDIT THIS AS NECESSARY if TIME-POINTS ARE ALTERED
tpsf = Reshape((xX,yY,8*64))(tpsf)
tpsf = SeparableConv2D(512, 1, padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)
tpsf = xCeptionblock_2D_BN(512, 1, tpsf)
tpsf = Conv2D(128, 1, padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)
tpsf = SeparableConv2D(256, 1, padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)
tpsf = resblock_2D_BN(256, 1, tpsf)

# # # # Individual branches for coercing coefficient-independent focus during learning.
# Alter this if coefficient number != 4

# Abundance coefficient #1
img1 = Conv2D(64, 1, padding='same', activation=None)(tpsf)
img1 = BatchNormalization()(img1)
img1 = Activation('relu')(img1)
img1 = Conv2D(32, 1, padding='same', activation=None)(img1)
img1 = BatchNormalization()(img1)
img1 = Activation('relu')(img1)
img1 = Conv2D(1, 1, padding='same', activation=None)(img1)
img1 = Activation('relu')(img1)

# Abundance coefficient #2
img2 = Conv2D(64, 1, padding='same', activation=None)(tpsf)
img2 = BatchNormalization()(img2)
img2 = Activation('relu')(img2)
img2 = Conv2D(32, 1, padding='same', activation=None)(img2)
img2 = BatchNormalization()(img2)
img2 = Activation('relu')(img2)
img2 = Conv2D(1, 1, padding='same', activation=None)(img2)
img2 = Activation('relu')(img2)

# Abundance coefficient #3
img3 = Conv2D(64, 1, padding='same', activation=None)(tpsf)
img3 = BatchNormalization()(img3)
img3 = Activation('relu')(img3)
img3 = Conv2D(32, 1, padding='same', activation=None)(img3)
img3 = BatchNormalization()(img3)
img3 = Activation('relu')(img3)
img3 = Conv2D(1, 1, padding='same', activation=None)(img3)
img3 = Activation('relu')(img3)

# Abundance coefficient #4
img4 = Conv2D(64, 1, padding='same', activation=None)(tpsf)
img4 = BatchNormalization()(img4)
img4 = Activation('relu')(img4)
img4 = Conv2D(32, 1, padding='same', activation=None)(img4)
img4 = BatchNormalization()(img4)
img4 = Activation('relu')(img4)
img4 = Conv2D(1, 1, padding='same', activation=None)(img4)
img4 = Activation('relu')(img4)

# construct new model
modelD = Model(inputs=t_data, outputs=[img1, img2, img3, img4])
rmsprop = RMSprop(lr=1e-5) # 1e-5 has worked well

# Compile the model
modelD.compile(loss='mse',
              optimizer=rmsprop,
              metrics=['mae'])

In [ ]:
c1 = np.expand_dims(c[:,:,:,0], axis=-1)
c2 = np.expand_dims(c[:,:,:,1], axis=-1)
c3 = np.expand_dims(c[:,:,:,2], axis=-1)
c4 = np.expand_dims(c[:,:,:,3], axis=-1)

where_are_NaNs = np.isnan(c1)
c1[where_are_NaNs] = 0
where_are_NaNs = np.isnan(c2)
c2[where_are_NaNs] = 0
where_are_NaNs = np.isnan(c3)
c3[where_are_NaNs] = 0
where_are_NaNs = np.isnan(c4)
c4[where_are_NaNs] = 0

earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience = 15, 
                              verbose = 0,
                              mode = 'auto')

fN = 'test' # specify name and directory for saving both best weights and loss/metrics
modelCheckPoint = ModelCheckpoint(filepath=fN+'.h5', 
                                  monitor='val_loss', 
                                  save_best_only=True, 
                                  verbose=0)

history = History()
csv_logger = CSVLogger(fN+'.log')
history = modelD.fit(tpsfD, [c1, c2, c3,c4], # Change this if coefficient numer != 4
          validation_split=0.2,
          batch_size=30, nb_epoch=100, verbose=1, shuffle=True, callbacks=[earlyStopping,csv_logger,modelCheckPoint])

In [ ]:
# Test dataset for trained model!

# Load data
t_data = 'C:/Users/jason/Dropbox/workingOn/unmixing/GitHub/UNMIX-ME_CNN/testD' # Training data directory

stacks = os.listdir(t_data)
numS = int(len(stacks))

# Pre-allocate for TPSF spectral data (time, x, y, channels)
tp = 256
x = 16
y = 16
ch = 16
tpsfD = np.ndarray(
        (numS, int(256), int(x), int(y), int(ch)), dtype=np.float32
        )
# Pre-allocate for abundance coefficients (time, x, y, channels)
nC = 4 # Number of coefficients
c = np.ndarray(
        (numS, int(nC), int(x), int(y)), dtype=np.float32
        )

i = 0;
for d in stacks:
    # Save values to respective mapping
    f = h5py.File(os.path.join(t_data,d),'r') 
    tpsfD[i,:,:,:,:] = f.get('tpsfs')
    f = h5py.File(os.path.join(t_data,d),'r') 
    c[i,:,:,:] = f.get('c')
    i = i + 1

# Move data to correct dimensionality/profile
tpsfD =  np.moveaxis(tpsfD, 1, -1)
tpsfD = np.moveaxis(tpsfD,1,-1)
c = np.moveaxis(c, 1, -1)

In [ ]:
# Same story as before - but for testing.
c1 = np.expand_dims(c[:,:,:,0], axis=-1)
c2 = np.expand_dims(c[:,:,:,1], axis=-1)
c3 = np.expand_dims(c[:,:,:,2], axis=-1)
c4 = np.expand_dims(c[:,:,:,3], axis=-1)
where_are_NaNs = np.isnan(c1)
c1[where_are_NaNs] = 0
where_are_NaNs = np.isnan(c2)
c2[where_are_NaNs] = 0
where_are_NaNs = np.isnan(c3)
c3[where_are_NaNs] = 0
where_are_NaNs = np.isnan(c4)
c4[where_are_NaNs] = 0

# Load weights of the trained model
modelD.load_weights(fN+'.h5')

# Use model on test data!
testV = modelD.predict(tpsfD)

In [ ]:
# Visualize results of all abundance coefficients (G.T versus UNMMIX-ME)
n = 0 # choose a particular data
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(4,2,1)
ax1.imshow(testV[0][n,:,:,0], interpolation='nearest', vmin=0, vmax=1.5)
ax2 = fig.add_subplot(4,2,2)
ax2.imshow(c1[n,:,:,0], interpolation='nearest', vmin=0, vmax=1.5)
ax3 = fig.add_subplot(4,2,3)
ax3.imshow(testV[1][n,:,:,0], interpolation='nearest', vmin=0, vmax=1.5)
ax4 = fig.add_subplot(4,2,4)
ax4.imshow(c2[n,:,:,0], interpolation='nearest', vmin=0, vmax=1.5)
ax5 = fig.add_subplot(4,2,5)
ax5.imshow(testV[2][n,:,:,0], interpolation='nearest', vmin=0, vmax=1.5)
ax6 = fig.add_subplot(4,2,6)
ax6.imshow(c3[n,:,:,0], interpolation='nearest', vmin=0, vmax=1.5)
ax7 = fig.add_subplot(4,2,7)
ax7.imshow(testV[3][n,:,:,0], interpolation='nearest', vmin=0, vmax=2)
ax8 = fig.add_subplot(4,2,8)
ax8.imshow(c4[n,:,:,0], interpolation='nearest', vmin=0, vmax=2)

In [ ]:
# # # # # # # # Save test data along with ground-truth into MATLAB data format for analysis!

import scipy.io as sio
sio.savemat(fN+'_testD.mat', {'cP':testV,
                                          'c1R': c1,
                                          'c2R': c2,
                                          'c3R': c3,
                                          'c4R': c4})